In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
#Load the ann trained model , scalar pivckle , onebot
model = load_model('model.h5')

#load the encoder and scalar 
with open ('one_hot_encoder_geo.pkl' , 'rb') as file :
    one_hot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scalar= pickle.load(file)

In [4]:
# example input data
input_data = {
        'creditscore ' : 600,
        'Geography': 'France',
        'Gender':'Male',
        'Age':40 ,
       ' Tenure' :3,
       'Balance':40000,
       'NumofProducts':2,
       'HasCrCard':1,
       'IsActiveMember':1,
       'EstimatedSalary':50000,
        
}

In [5]:
#one-hot encode 'geography'
geo_encoded= one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns = one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\gaura\anaconda3\envs\tf_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,creditscore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,40000,2,1,1,50000


In [7]:
##encode categorial variables
input_df ['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,creditscore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,40000,2,1,1,50000


In [8]:
##concatenation one hot encoded
input_df = pd.concat([input_df.drop("Geography", axis=1),geo_encoded_df], axis=1)
input_df

,creditscore,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,40000,2,1,1,50000,1.0,0.0,0.0


In [9]:
from sklearn.preprocessing import StandardScaler

# Ensure consistent feature order
expected_features = ['CreditScore', 'NumOfProducts', 'Tenure']
for col in expected_features:
    if col not in input_df.columns:
        input_df[col] = 0

# Fit and transform using the same feature set
scaler = StandardScaler()
scaler.fit(input_df[expected_features])

input_scaled = scaler.transform(input_df[expected_features])

In [16]:
#predict churn 
prediction = model.predict(input_scaled)
prediction

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 12, but received input with shape (1, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 3), dtype=float32)
  • training=False
  • mask=None

In [14]:
prediction_probab = prediction[0][0]

NameError: name 'prediction' is not defined